In [1]:
import keras
keras.__version__

'2.14.0'

In [2]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) =reuters.load_data(num_words=10000)

In [3]:
len(train_data)

8982

In [4]:
len(test_data)

2246

In [5]:
train_data[10]

[1,
 245,
 273,
 207,
 156,
 53,
 74,
 160,
 26,
 14,
 46,
 296,
 26,
 39,
 74,
 2979,
 3554,
 14,
 46,
 4689,
 4329,
 86,
 61,
 3499,
 4795,
 14,
 61,
 451,
 4329,
 17,
 12]

In [9]:
word_index: dict = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decode_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
print(decode_newswire)

? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [10]:
train_labels[10]

3

In [15]:
import numpy as np

def vectorize_sequences(sequences, dimenstion=10000):
    results = np.zeros((len(sequences), dimenstion))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [17]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [18]:
from keras.utils import to_categorical
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [167]:
from keras import models, layers
actReg = keras.layers.ActivityRegularization(
    input_shape=(10000,), l1=4.0, l2=6.0)
model = models.Sequential()
model.add(actReg)
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [168]:
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [169]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [170]:
history = model.fit(partial_x_train, partial_y_train, epochs=9,
                    batch_size=128,
                    validation_data=(x_val, y_val))

Epoch 1/9
63/63 [==============================] - 1s 12ms/step - loss: 817.5926 - accuracy: 0.6017 - val_loss: 825.0411 - val_accuracy: 0.7040
Epoch 2/9
63/63 [==============================] - 1s 10ms/step - loss: 816.7773 - accuracy: 0.7725 - val_loss: 824.8388 - val_accuracy: 0.7460
Epoch 3/9
63/63 [==============================] - 1s 11ms/step - loss: 816.4386 - accuracy: 0.8468 - val_loss: 824.7856 - val_accuracy: 0.7740
Epoch 4/9
63/63 [==============================] - 1s 12ms/step - loss: 816.2295 - accuracy: 0.8970 - val_loss: 824.6819 - val_accuracy: 0.7980
Epoch 5/9
63/63 [==============================] - 1s 13ms/step - loss: 816.0916 - accuracy: 0.9270 - val_loss: 824.7650 - val_accuracy: 0.7900
Epoch 6/9
63/63 [==============================] - 1s 11ms/step - loss: 816.0239 - accuracy: 0.9404 - val_loss: 824.7006 - val_accuracy: 0.8140
Epoch 7/9
63/63 [==============================] - 1s 12ms/step - loss: 815.9807 - accuracy: 0.9473 - val_loss: 824.7211 - val_accuracy:

In [173]:
predictions = model.predict(x_test)

71/71 [==============================] - 0s 2ms/step


In [177]:
sum = 0
for i in range(len(predictions)):
    if np.argmax(predictions[i]) == test_labels[i]:
        sum +=1
sum = sum / len(predictions) * 100
print(f"true prediction: {sum}%")

true prediction: 77.51558325912734%
